## load rvest library

In [18]:
library(rvest)
library(httr)

## Extract bike sharing system tables and convert it into dataframe

In [43]:
url <-"https://en.wikipedia.org/wiki/List_of_bicycle-sharing_systems"
root_node<- read_html(url)
table_node<- html_nodes(root_node,"table")
bike_sharing<-html_table(table_node[[2]],header=TRUE)
head(bike_sharing,10)

Country,City,Name,System,Operator,Launched,Discontinued,Stations,Bicycles,Daily ridership
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Albania,Tirana[5],Ecovolis,,,March 2011,,8,200,
Argentina,Buenos Aires[6][7],Ecobici,Serttel Brasil[8],Bike In Baires Consortium.[9],2010,,400,4000,21917
Argentina,Mendoza[10],Metrobici,,,2014,,2,40,
Argentina,Rosario,Mi Bici Tu Bici[11],,,2 December 2015,,47,480,
Argentina,"San Lorenzo, Santa Fe",Biciudad,Biciudad,,27 November 2016,,8,80,
Australia,Melbourne[12],Melbourne Bike Share,PBSC & 8D,Motivate,June 2010,30 November 2019[13],53,676,
Australia,Melbourne[12],oBike,4 Gen. oBike,,July 2017,July 2018,dockless,1250,
Australia,Brisbane[14][15],CityCycle,3 Gen. Cyclocity,JCDecaux,September 2010,,150,2000,
Australia,Sydney,oBike,4 Gen. oBike,,July 2017,July 2018,dockless,1250,


## Export the bike sharing system dataframe

In [44]:
#export the bike sharing system dataframe
write.csv(bike_sharing,file='raw_bike_sharing_system.csv',row.names=FALSE)

## Create some empty vectors to hold data temporarily 

In [45]:
# Create some empty vectors to hold data temporarily

# City name column
city <- c()
# Weather column, rainy or cloudy, etc
weather <- c()
# Sky visibility column
visibility <- c()
# Current temperature column
temp <- c()
# Max temperature column
temp_min <- c()
# Min temperature column
temp_max <- c()
# Pressure column
pressure <- c()
# Humidity column
humidity <- c()
# Wind speed column
wind_speed <- c()
# Wind direction column
wind_deg <- c()
# Forecast timestamp
forecast_datetime <- c()
# Season column
# Note that for season, you can hard code a season value from levels Spring, Summer, Autumn, and Winter based on your current month.
season <- c()

## get 5 day weather forecast using OpenWeather API

In [46]:
your_api_key <- "5bdb8ec6aaea3bbcca8225f64649d6d0"

In [47]:
# Get forecast data for a given city list
    get_weather_forecaset_by_cities <- function(city_names){
        for (city_name in city_names){
          forecast_df <- data.frame()
        # Forecast API URL
        forecast_url <- 'https://api.openweathermap.org/data/2.5/forecast'
        # Create query parameters
        forecast_query <- list(q = city_name, appid = your_api_key, units="metric")
        # Make HTTP GET call for the given city
        response <- GET(forecast_url,query= forecast_query)
        # Note that the 5-day forecast JSON result is a list of lists. You can print the reponse to check the results
        # results <- json_list$list
        json_list<- content(response,as="parsed")
       results <- json_list$list
        # Loop the json result
      for(result in results) { 
          city <- c(city,city_names)
          weather <- c(weather,result$weather[[1]]$main)
          visibility<-c(visibility,result$visibility)
          temp<- c(temp,result$main$temp)
          temp_min <-c(temp_min,result$main$temp_min)
          temp_max <-c(temp_max,result$main$temp_max)
         pressure <- c(pressure,result$main$pressure)
          humidity <- c(humidity,result$main$humidity)
          wind_speed <- c(wind_speed,result$wind$speed)
          wind_deg <-c(wind_deg,result$wind$deg)
          forecast_datetime <-c(forecast_datetime,result$dt_txt) 
         } 
        # Add the R Lists into a data frame
      forecast_df<- data.frame( city=city,
                                 weather=weather,
                                 visibility=visibility,
                                 temp=temp,
                                 temp_min=temp_min,
                                 temp_max=temp_max,
                                 pressure=pressure,
                                 humidity=humidity,
                                 wind_speed=wind_speed,
                                 wind_deg=wind_deg,
                                 forecast=forecast_datetime)
      }
   # get_city
    # Return a data frame
      return(forecast_df)
}
city_names <- c("Seoul", "Washington","D.C.", "Paris", "Suzhou")
cities_weather_df <- get_weather_forecaset_by_cities(city_names)
head(cities_weather_df,10)

,city,weather,visibility,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,forecast
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<chr>
1,Seoul,Clear,10000,23.96,23.86,23.96,1007,83,0.90,179,2022-08-18 15:00:00
2,Washington,Clouds,10000,23.53,22.66,23.53,1007,84,1.35,174,2022-08-18 18:00:00
3,D.C.,Clouds,10000,23.13,22.71,23.13,1006,84,1.32,135,2022-08-18 21:00:00
4,Paris,Clouds,10000,24.27,24.27,24.27,1005,77,2.31,160,2022-08-19 00:00:00
5,Suzhou,Clouds,10000,26.02,26.02,26.02,1004,65,3.14,199,2022-08-19 03:00:00
6,Seoul,Rain,9054,23.92,23.92,23.92,1002,91,2.08,155,2022-08-19 06:00:00
7,Washington,Rain,10000,24.99,24.99,24.99,1001,96,4.77,212,2022-08-19 09:00:00
8,D.C.,Rain,10000,25.08,25.08,25.08,1002,95,3.53,251,2022-08-19 12:00:00
9,Paris,Rain,10000,24.90,24.90,24.90,1001,98,2.15,238,2022-08-19 15:00:00


### Export the cities_weather_forecast as csv

In [48]:
write.csv(cities_weather_df, "raw_cities_weather_forecast.csv", row.names=FALSE)

### Download  General City information dataset

In [49]:
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_worldcities.csv"
download.file(url, destfile = "raw_worldcities.csv")

### Download Hourly bike sharing demand dataset

In [50]:
url <- "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-RP0321EN-SkillsNetwork/labs/datasets/raw_seoul_bike_sharing.csv"
download.file(url, destfile = "raw_seoul_bike_sharing.csv")